In [1]:
from icecube import dataio, icetray , dataclasses 


In [2]:
import pandas as pd
pd.set_option("display.max_columns", None)  


In [3]:
from collections.abc import Iterable

In [4]:
GCD_PATH = "/project/6008051/pone_simulation/GCD_Library/PONE_800mGrid.i3.gz"
data_file = dataio.I3File(GCD_PATH)
data_file.pop_frame()
data_file.pop_frame()
detector_status = data_file.pop_frame()

In [5]:
# dir(detector_status)

In [6]:
print(type(detector_status))
print("Stop:", detector_status.Stop)   
print("Key's:", list(detector_status.keys()))


<class 'icecube._icetray.I3Frame'>
Stop: DetectorStatus
Key's: ['EndTime', 'I3ModuleGeoMap', 'I3OMGeoMap', 'Subdetectors', 'BadDomsList', 'BadDomsListSLC', 'I3Calibration', 'StartTime', 'I3DetectorStatus', 'I3Geometry', 'SPEAbove', 'SPEScalingFactors']


In [7]:
# Small helper: print the type and a short summary for each key in the frame
for key in detector_status.keys():
    obj = detector_status[key]
    print("="*60)
    print(f"Key: {key}")
    print("Type:", type(obj))
    try:
        print("dir(obj) first 10:", dir(obj)[:10])
    except Exception as e:
        print("dir(obj) could not be obtained:", e)

Key: EndTime
Type: <class 'icecube._dataclasses.I3Time'>
dir(obj) first 10: ['Apr', 'Aug', 'BadMonth', 'BadWeekday', 'Dec', 'Feb', 'Friday', 'Jan', 'Jul', 'Jun']
Key: I3ModuleGeoMap
Type: <class 'icecube._dataclasses.I3ModuleGeoMap'>
dir(obj) first 10: ['__class__', '__contains__', '__copy__', '__deepcopy__', '__delattr__', '__delitem__', '__dict__', '__dir__', '__doc__', '__eq__']
Key: I3OMGeoMap
Type: <class 'icecube._dataclasses.I3OMGeoMap'>
dir(obj) first 10: ['__class__', '__contains__', '__copy__', '__deepcopy__', '__delattr__', '__delitem__', '__dict__', '__dir__', '__doc__', '__eq__']
Key: Subdetectors
Type: <class 'icecube._dataclasses.I3MapModuleKeyString'>
dir(obj) first 10: ['__class__', '__contains__', '__copy__', '__deepcopy__', '__delattr__', '__delitem__', '__dict__', '__dir__', '__doc__', '__eq__']
Key: BadDomsList
Type: <class 'icecube._dataclasses.I3VectorOMKey'>
dir(obj) first 10: ['__class__', '__contains__', '__copy__', '__deepcopy__', '__delattr__', '__delitem__'

WARN (dataclasses): Reading version 0 from file into version 1 of I3ModuleGeo class, all modules are assumed spherical. (I3ModuleGeo.cxx:75 in void I3ModuleGeo::serialize(Archive&, unsigned int) [with Archive = icecube::archive::portable_binary_iarchive])
WARN (dataclasses): Reading version 0 from file into version 1 of I3ModuleGeo class, all modules are assumed spherical. (I3ModuleGeo.cxx:75 in void I3ModuleGeo::serialize(Archive&, unsigned int) [with Archive = icecube::archive::portable_binary_iarchive])
WARN (dataclasses): Reading version 0 from file into version 1 of I3ModuleGeo class, all modules are assumed spherical. (I3ModuleGeo.cxx:75 in void I3ModuleGeo::serialize(Archive&, unsigned int) [with Archive = icecube::archive::portable_binary_iarchive])
WARN (dataclasses): Reading version 0 from file into version 1 of I3ModuleGeo class, all modules are assumed spherical. (I3ModuleGeo.cxx:75 in void I3ModuleGeo::serialize(Archive&, unsigned int) [with Archive = icecube::archive::por

Key: I3Calibration
Type: <class 'icecube._dataclasses.I3Calibration'>
dir(obj) first 10: ['__class__', '__copy__', '__deepcopy__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__']
Key: StartTime
Type: <class 'icecube._dataclasses.I3Time'>
dir(obj) first 10: ['Apr', 'Aug', 'BadMonth', 'BadWeekday', 'Dec', 'Feb', 'Friday', 'Jan', 'Jul', 'Jun']
Key: I3DetectorStatus
Type: <class 'icecube._dataclasses.I3DetectorStatus'>
dir(obj) first 10: ['__class__', '__copy__', '__deepcopy__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__']
Key: I3Geometry
Type: <class 'icecube._dataclasses.I3Geometry'>
dir(obj) first 10: ['__class__', '__copy__', '__deepcopy__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__']
Key: SPEAbove
Type: <class 'icecube._dataclasses.I3MapKeyDouble'>
dir(obj) first 10: ['__class__', '__contains__', '__copy__', '__deepcopy__', '__delattr__', '__delitem__', '__dict__', '__dir__'

# - I3Calibration
###### This may be included in G or C frame. Check it

# - StartTime and EndTime (same as Geometry.StartTime and Geometry.EndTime)


In [8]:
# start = detector_status["StartTime"]
# end   = detector_status["EndTime"]
# print(start)
# print(end)

In [9]:
# start

In [10]:
# end

In [11]:
# print(repr(start))
# print(str(start))
# 

In [12]:
# print(repr(end))
# print(str(end))
# 

# - DetectorStatus.I3DetectorStatus

In [13]:
det_obj = detector_status["I3DetectorStatus"]

print("Type:", type(det_obj))




Type: <class 'icecube._dataclasses.I3DetectorStatus'>


In [14]:
# --- public attribute's ---
attrs = [a for a in dir(det_obj) if not a.startswith("_")]
print("\nPublic attrs:")
for a in attrs:
    print("  -", a)


Public attrs:
  - daq_configuration_name
  - dom_status
  - end_time
  - from_frame
  - start_time
  - trigger_status


In [15]:

rows = []
for a in attrs:
    val = getattr(det_obj, a)
    try:
        n = len(val)        
    except TypeError:
        n = None

    rows.append({
        "attr": a,
        "type": type(val).__name__,
        "len": n,
        "repr_sample": repr(val)[:80],   
    })

det_status_i3_df = pd.DataFrame(rows)
det_status_i3_df


,attr,type,len,repr_sample
0,daq_configuration_name,str,14.0,'P-ONE_Estimate'
1,dom_status,Map_OMKey_I3DOMStatus,108800.0,<icecube._dataclasses.Map_OMKey_I3DOMStatus ob...
2,end_time,I3Time,NaN,"I3Time(2039, 0L)"
3,from_frame,function,NaN,<Boost.Python.function object at 0x1c908a0>
4,start_time,I3Time,NaN,"I3Time(2019, 0L)"
5,trigger_status,Map_TriggerKey_I3TriggerStatus,6.0,<icecube._dataclasses.Map_TriggerKey_I3Trigger...


## -- dom_status

In [16]:
dom_status_map = det_obj.dom_status

print("Type:", type(dom_status_map))
print("Number of entries:", len(dom_status_map))

# peek at one exam ple
first_key, first_val = next(iter(dom_status_map.items()))
print("\nExample key:", first_key)
print("Example value type:", type(first_val))
print("Example value repr:\n", first_val)


Type: <class 'icecube._dataclasses.Map_OMKey_I3DOMStatus'>
Number of entries: 108800

Example key: OMKey(1,1,1)
Example value type: <class 'icecube._dataclasses.I3DOMStatus'>
Example value repr:


In [17]:

dom_attrs = [a for a in dir(first_val) if not a.startswith("_")]

rows = []
for a in dom_attrs:
    v = getattr(first_val, a)
    try:
        n = len(v)
    except TypeError:
        n = None

    rows.append({
        "attr": a,
        "type": type(v).__name__,
        "len": n,
        "repr": repr(v)[:80],  
    })

dom_status_one_df = pd.DataFrame(rows).sort_values("attr")
dom_status_one_df


,attr,type,len,repr
0,CPU,TrigMode,None,icecube._dataclasses.TrigMode.CPU
1,CableType,type,None,<class 'icecube._dataclasses.CableType'>
2,DOMGain,type,None,<class 'icecube._dataclasses.DOMGain'>
3,Down,LCMode,None,icecube._dataclasses.LCMode.Down
4,Flasher,TrigMode,None,icecube._dataclasses.TrigMode.Flasher
5,High,DOMGain,None,icecube._dataclasses.DOMGain.High
6,LCMode,type,None,<class 'icecube._dataclasses.LCMode'>
7,LCOff,LCMode,None,icecube._dataclasses.LCMode.LCOff
8,Low,DOMGain,None,icecube._dataclasses.DOMGain.Low
9,MPE,TrigMode,None,icecube._dataclasses.TrigMode.MPE


In [18]:
import inspect

example_status = first_val   # this is the I3DOMStatus for OMKey(1,1,1)

per_dom_attrs = []
for a in dir(example_status):
    if a.startswith("_"):
        continue
    # only attributes starting with a lowercase letter (skip enum constants like CPU, SPE, etc.)
    if not a[0].islower():
        continue

    v = getattr(example_status, a)
    # skip methods / functions
    if inspect.ismethod(v) or inspect.isfunction(v):
        continue

    per_dom_attrs.append(a)

print("Per-DOM attributes:")
for a in per_dom_attrs:
    print("  -", a)


Per-DOM attributes:
  - cable_type
  - dac_fadc_ref
  - dac_trigger_bias_0
  - dac_trigger_bias_1
  - delta_compress
  - dom_gain_type
  - fb_brightness
  - fb_delay
  - fb_length
  - fb_mask
  - fb_rate
  - fb_state
  - fe_pedestal
  - lc_mode
  - lc_span
  - lc_window_post
  - lc_window_pre
  - mpe_threshold
  - n_bins_atwd_0
  - n_bins_atwd_1
  - n_bins_atwd_2
  - n_bins_atwd_3
  - n_bins_fadc
  - pmt_hv
  - slc_active
  - spe_threshold
  - status_atwd_a
  - status_atwd_b
  - status_fadc
  - trig_mode
  - tx_mode


In [19]:
import pandas as pd

rows = []

for omkey, status in dom_status_map.items():
    row = {
        "string": omkey.string,
        "om": omkey.om,
        "pmt": getattr(omkey, "pmt", None),
    }

    for a in per_dom_attrs:
        v = getattr(status, a)

        # enum'ları okunur stringe çevir
        if type(v).__name__ in ["DOMGain", "LCMode", "TrigMode", "OnOff", "CableType"]:
            row[a] = v.name
        else:
            row[a] = v

    rows.append(row)

dom_status_df = pd.DataFrame(rows)
dom_status_df.head()


,string,om,pmt,cable_type,dac_fadc_ref,dac_trigger_bias_0,dac_trigger_bias_1,delta_compress,dom_gain_type,fb_brightness,fb_delay,fb_length,fb_mask,fb_rate,fb_state,fe_pedestal,lc_mode,lc_span,lc_window_post,lc_window_pre,mpe_threshold,n_bins_atwd_0,n_bins_atwd_1,n_bins_atwd_2,n_bins_atwd_3,n_bins_fadc,pmt_hv,slc_active,spe_threshold,status_atwd_a,status_atwd_b,status_fadc,trig_mode,tx_mode
0,1,1,1,UnknownCableType,800.0,851.427461,851.315322,On,High,2147483647,2147483647,2147483647,2147483647,2147483647,Unknown,2130.0,UpOrDown,2,1000.0,1000.0,652.225759,0,0,0,0,0,0.000001,True,565.908216,On,On,On,SPE,UpAndDown
1,1,1,2,UnknownCableType,800.0,851.427461,851.315322,On,High,2147483647,2147483647,2147483647,2147483647,2147483647,Unknown,2130.0,UpOrDown,2,1000.0,1000.0,652.225759,0,0,0,0,0,0.000001,True,565.908216,On,On,On,SPE,UpAndDown
2,1,1,3,UnknownCableType,800.0,851.427461,851.315322,On,High,2147483647,2147483647,2147483647,2147483647,2147483647,Unknown,2130.0,UpOrDown,2,1000.0,1000.0,652.225759,0,0,0,0,0,0.000001,True,565.908216,On,On,On,SPE,UpAndDown
3,1,1,4,UnknownCableType,800.0,851.427461,851.315322,On,High,2147483647,2147483647,2147483647,2147483647,2147483647,Unknown,2130.0,UpOrDown,2,1000.0,1000.0,652.225759,0,0,0,0,0,0.000001,True,565.908216,On,On,On,SPE,UpAndDown
4,1,1,5,UnknownCableType,800.0,851.427461,851.315322,On,High,2147483647,2147483647,2147483647,2147483647,2147483647,Unknown,2130.0,UpOrDown,2,1000.0,1000.0,652.225759,0,0,0,0,0,0.000001,True,565.908216,On,On,On,SPE,UpAndDown


In [20]:
n_unique = dom_status_df.nunique()

const_cols = n_unique[n_unique == 1].index.tolist()
vary_cols  = n_unique[n_unique > 1].index.tolist()

print("Columns that are constant across all DOMs:")
for c in const_cols:
    print("  -", c)

print("\nColumns that vary between DOMs:")
for c in vary_cols:
    print("  -", c)


Columns that are constant across all DOMs:
  - cable_type
  - dac_fadc_ref
  - dac_trigger_bias_0
  - dac_trigger_bias_1
  - delta_compress
  - dom_gain_type
  - fb_brightness
  - fb_delay
  - fb_length
  - fb_mask
  - fb_rate
  - fb_state
  - fe_pedestal
  - lc_mode
  - lc_span
  - lc_window_post
  - lc_window_pre
  - mpe_threshold
  - n_bins_atwd_0
  - n_bins_atwd_1
  - n_bins_atwd_2
  - n_bins_atwd_3
  - n_bins_fadc
  - pmt_hv
  - slc_active
  - spe_threshold
  - status_atwd_a
  - status_atwd_b
  - status_fadc
  - trig_mode
  - tx_mode

Columns that vary between DOMs:
  - string
  - om
  - pmt


### 3.3.5 trigger_status

In [21]:

trig_status_map = det_obj.trigger_status

print("Type:", type(trig_status_map))
print("Number of entries:", len(trig_status_map))

# peek at one example
first_tkey, first_tval = next(iter(trig_status_map.items()))
print("\nExample TriggerKey:", first_tkey)
print("Example value type:", type(first_tval))
print("Example value repr:\n", first_tval)


Type: <class 'icecube._dataclasses.Map_TriggerKey_I3TriggerStatus'>
Number of entries: 6

Example TriggerKey: [IN_ICE:SIMPLE_MULTIPLICITY:1006]
Example value type: <class 'icecube._dataclasses.I3TriggerStatus'>
Example value repr:


In [22]:


first_tkey, first_tval = next(iter(trig_status_map.items()))

trig_attrs = [a for a in dir(first_tval) if not a.startswith("_")]

rows = []
for a in trig_attrs:
    v = getattr(first_tval, a)
    try:
        n = len(v)
    except TypeError:
        n = None
    rows.append({
        "attr": a,
        "type": type(v).__name__,
        "len": n,
        "repr": repr(v)[:80],
    })

trig_status_one_df = pd.DataFrame(rows).sort_values("attr")
trig_status_one_df


,attr,type,len,repr
0,ALL,Subdetector,NaN,icecube._dataclasses.Subdetector.ALL
1,I3TriggerReadoutConfig,class,NaN,<class 'icecube._dataclasses.I3TriggerReadoutC...
2,ICETOP,Subdetector,NaN,icecube._dataclasses.Subdetector.ICETOP
3,INICE,Subdetector,NaN,icecube._dataclasses.Subdetector.INICE
4,NOT_SPECIFIED,Subdetector,NaN,icecube._dataclasses.Subdetector.NOT_SPECIFIED
5,Subdetector,type,NaN,<class 'icecube._dataclasses.Subdetector'>
6,identity,method,NaN,<bound method identity of <icecube._dataclasse...
7,readout_settings,map_Subdetector_I3TriggerReadoutConfig,2.0,<icecube._dataclasses.map_Subdetector_I3Trigge...
8,trigger_name,str,21.0,'SimpleMajorityTrigger'
9,trigger_settings,TestMapStringString,2.0,<icecube._dataclasses.TestMapStringString obje...


In [23]:

first_tkey, first_tval = next(iter(trig_status_map.items()))

per_trig_attrs = []
for a in dir(first_tval):
    if a.startswith("_"):
        continue
    if not a[0].islower():
        continue

    v = getattr(first_tval, a)
    if inspect.ismethod(v) or inspect.isfunction(v):
        continue

    per_trig_attrs.append(a)

print("Per-trigger attrs:")
for a in per_trig_attrs:
    print("  -", a)


Per-trigger attrs:
  - readout_settings
  - trigger_name
  - trigger_settings


In [24]:
rows = []

for tkey, status in trig_status_map.items():
    row = {
        # Info from the TriggerKey
        "source":    tkey.source,      # e.g. IN_ICE, GLOBAL, ICETOP
        "type":      tkey.type,        # e.g. SIMPLE_MULTIPLICITY
        "config_id": tkey.config_id,   # e.g. 1006
    }

    # Make enums human-readable strings (source/type are usually enums)
    for k in ["source", "type"]:
        v = row[k]
        if hasattr(v, "name"):
            row[k] = v.name

    # Per-trigger attributes
    for a in per_trig_attrs:
        v = getattr(status, a)

        # For complex maps like readout_settings / trigger_settings,
        # convert to something printable; try dict, fall back to repr
        if hasattr(v, "items"):
            try:
                row[a] = dict(v.items())
            except Exception:
                row[a] = repr(v)
        else:
            row[a] = v

    rows.append(row)

trig_status_df = pd.DataFrame(rows)
trig_status_df


,source,type,config_id,readout_settings,trigger_name,trigger_settings
0,IN_ICE,SIMPLE_MULTIPLICITY,1006,{ICETOP: <icecube._dataclasses.I3TriggerReadou...,SimpleMajorityTrigger,<icecube._dataclasses.TestMapStringString obje...
1,IN_ICE,SIMPLE_MULTIPLICITY,1011,{ICETOP: <icecube._dataclasses.I3TriggerReadou...,SimpleMajorityTrigger,<icecube._dataclasses.TestMapStringString obje...
2,IN_ICE,SLOW_PARTICLE,24002,{ICETOP: <icecube._dataclasses.I3TriggerReadou...,SlowMPTrigger,<icecube._dataclasses.TestMapStringString obje...
3,IN_ICE,STRING,1007,{ICETOP: <icecube._dataclasses.I3TriggerReadou...,ClusterTrigger,<icecube._dataclasses.TestMapStringString obje...
4,IN_ICE,VOLUME,21001,{ICETOP: <icecube._dataclasses.I3TriggerReadou...,CylinderTrigger,<icecube._dataclasses.TestMapStringString obje...
5,ICE_TOP,SIMPLE_MULTIPLICITY,102,{ALL: <icecube._dataclasses.I3TriggerReadoutCo...,SimpleMajorityTrigger,<icecube._dataclasses.TestMapStringString obje...


In [25]:
# pick one trigger, e.g. the first row
example_key, example_status = next(iter(trig_status_map.items()))

print("TriggerKey:", example_key)
print("\ntrigger_name:", example_status.trigger_name)

# --- trigger_settings: use string repr instead of .items() ---
print("\ntrigger_settings (as text):")
print(str(example_status.trigger_settings))

# If you want it line-by-line:
print("\ntrigger_settings (line by line):")
for line in str(example_status.trigger_settings).splitlines():
    print(" ", line)

# --- readout_settings: this one *does* support .items() ---
print("\nreadout_settings (per subdetector):")
for subd, cfg in example_status.readout_settings.items():
    print(" ", subd, "->", cfg)


TriggerKey: [IN_ICE:SIMPLE_MULTIPLICITY:1006]

trigger_name: SimpleMajorityTrigger

trigger_settings (as text):

trigger_settings (line by line):

readout_settings (per subdetector):
  ICETOP -> <icecube._dataclasses.I3TriggerReadoutConfig object at 0x14a8d4549fc0>
  INICE -> <icecube._dataclasses.I3TriggerReadoutConfig object at 0x14a8d454a180>


# - I3Geometry (same as Geometry.I3Geometry)


In [26]:
# geo = detector_status["I3Geometry"]
# 
# print("Type:", type(geo))
# print("Attributes:")
# print(dir(geo))
# 

In [27]:
# print(geo)

In [28]:
# # All attributes
# all_attrs = dir(geo)
# 
# # Non-dunder attributes
# public_attrs = [a for a in all_attrs if not a.startswith("_")]
# 
# print("Public attrs:")
# for a in public_attrs:
#     print("  -", a)
# 

In [29]:
# rows = []
# for a in public_attrs:
#     value = getattr(geo, a)
#     # try to get length if it behaves like a container
#     try:
#         n = len(value)
#     except TypeError:
#         n = None
# 
#     rows.append({
#         "attr": a,
#         "type": type(value).__name__,
#         "len": n,
#         "repr_sample": repr(value),  
#     })
# 
# geo_attrs_df = pd.DataFrame(rows)
# geo_attrs_df
# 

In [30]:
# for name in public_attrs:
#     val = getattr(geo, name)
# 
#     print(f"\n{name}")
#     print("  type:", type(val))
# 
#     # try to show some size info
#     if hasattr(val, "keys"):
#         print("  n_keys:", len(val))
#     elif hasattr(val, "__len__") and not isinstance(val, (str, bytes)):
#         print("  len:", len(val))
# 
#     # small repr preview
#     print("  repr:", repr(val)[:200])
# 

In [31]:
# om_map  = detector_status["I3OMGeoMap"]
# geo     = detector_status["I3Geometry"]
# om_map2 = geo.omgeo
# 
# print(len(om_map), len(om_map2))
# 
# print("same keys? :", set(om_map.keys()) == set(om_map2.keys()))
# 
# keys = list(om_map.keys())
# print("all values equal? :",
#       all(om_map[k] == om_map2[k] for k in keys))
# 

In [32]:
# print(om_map is om_map2)


# - SPEAbove (same as Calibration.SPEAbove)

In [33]:
# spe_above = detector_status["SPEAbove"]
# 
# print("Type:", type(spe_above))

In [34]:
# try:
#     print("Number of entries:", len(spe_above))
# except TypeError:
#     print("len() not defined")
# 
# # peek at a few entries
# print("\nFirst few entries:")
# for i, (k, v) in enumerate(spe_above.items()):
#     print("  key:", k, "   value:", v)

In [35]:
# print(spe_above)

In [36]:
# if len(spe_above) == 1:
#     (only_key, only_val), = spe_above.items()
#     print("Single entry in SPEAbove:")
#     print("  key  :", only_key)
#     print("  value:", only_val)
# else:
#     print("SPEAbove has", len(spe_above), "entries.")
# 
# 
# spe_above_df = pd.DataFrame(
#     [{"string": only_key.string,
#       "om": only_key.om,
#       "pmt": getattr(only_key, "pmt", None),
#       "value": only_val}]
# )
# spe_above_df
# 

# - SPEScalingFactors (same as Calibration.SPEScalingFactors)


In [37]:
# spe_scale = detector_status["SPEScalingFactors"]
# 
# print("Type:", type(spe_scale))

In [38]:
# try:
#     print("Number of entries:", len(spe_scale))
# except TypeError:
#     print("len() not defined")
# 
# print("\nFirst few entries:")
# for i, (k, v) in enumerate(spe_scale.items()):
#     print("  key:", k, "   value:", v)
# 

In [39]:
# rows = []
# for k, v in spe_scale.items():
#     rows.append({
#         "string": getattr(k, "string", None),
#         "om": getattr(k, "om", None),
#         "pmt": getattr(k, "pmt", None),
#         "key_repr": str(k),
#         "value": v,
#     })
# 
# spe_scale_df = pd.DataFrame(rows)
# spe_scale_df.head()

# - I3ModuleGeoMap (same as Geometry.I3ModuleGeoMap)


In [40]:
# mod_map = detector_status["I3ModuleGeoMap"]
# print(type(mod_map))
# print("Number of modules:", len(mod_map))
# 
# # 340 * 20

In [41]:
# print(mod_map)

In [42]:
# # discover attrs once
# k0, g0 = next(iter(mod_map.items()))
# key_attrs = [a for a in dir(k0) if not a.startswith("_")]
# geo_attrs = [a for a in dir(g0) if not a.startswith("_")]
# 
# rows = []
# 
# for mkey, mgeo in mod_map.items():
#     # try both 'pos' and 'position'
#     pos = getattr(mgeo, "pos", None)
#     if pos is None:
#         pos = getattr(mgeo, "position", None)
# 
#     ori = getattr(mgeo, "orientation", None)
#     d   = getattr(mgeo, "dir", None)
# 
#     row = {}
# 
#     # ---- ModuleKey info ----
#     for a in key_attrs:
#         v = getattr(mkey, a)
#         if isinstance(v, (int, float, str, bool)):
#             row[f"key_{a}"] = v
# 
#     # ---- position ----
#     if pos is not None:
#         row["x"] = pos.x
#         row["y"] = pos.y
#         row["z"] = pos.z
# 
#     # ---- direction vector ----
#     if d is not None:
#         row["dir_x"] = d.x
#         row["dir_y"] = d.y
#         row["dir_z"] = d.z
#         # if zenith/azimuth exist:
#         for name in ("zenith", "azimuth"):
#             if hasattr(d, name):
#                 row[f"dir_{name}"] = getattr(d, name)
# 
#     # ---- orientation vectors ----
#     if ori is not None:
#         for base, vec in (("up", ori.up), ("right", ori.right)):
#             row[f"{base}_x"] = vec.x
#             row[f"{base}_y"] = vec.y
#             row[f"{base}_z"] = vec.z
# 
#     # ---- all other scalar attrs of mgeo ----
#     for a in geo_attrs:
#         if a in {"pos", "position", "orientation", "dir"}:
#             continue
#         if a in row:  # don't overwrite existing keys
#             continue
#         v = getattr(mgeo, a)
#         if isinstance(v, (int, float, str, bool)):
#             row[a] = v
# 
#     rows.append(row)
# 
# mod_df = pd.DataFrame(rows)
# mod_df.head()
# 

In [43]:
# mod_df.shape        


In [44]:
# unique_counts = mod_df.nunique(dropna=False)
# 
# constant_cols = unique_counts[unique_counts == 1].index.tolist()
# varying_cols  = unique_counts[unique_counts > 1].index.tolist()
# 
# print("Columns that are constant across all rows:")
# for c in constant_cols:
#     print("  -", c)
# 
# print("\nColumns that vary:")
# for c in varying_cols:
#     print("  -", c)
# 

In [45]:
# mod_df = mod_df.loc[:, mod_df.nunique(dropna=False) > 1]


In [46]:
# mod_df

# - I3OMGeoMap (same as Geometry.I3OMGeoMap)¶


In [47]:
# om_map = detector_status["I3OMGeoMap"]
# print(type(om_map))
# print("Number of OMs:", len(om_map))
# 
# # 340 * 20 * 16

In [48]:
# om_map.keys()
# OMKey has information of String Id, OM Id, PMT Id

In [49]:
# for k in list(om_map.keys())[:5]:
#    print(k)

In [50]:
# for k in list(om_map.values())[:5]:
#    print(k)


In [51]:
# sample_key, sample_geo = next(iter(om_map.items()))
# attrs = [a for a in dir(sample_geo) if not a.startswith("_")]
# 
# # we handle these manually 
# skip = {"position", "orientation", "omtype", "area"}
# attrs = [a for a in attrs if a not in skip]
# 
# rows = []
# 
# for omkey, omgeo in om_map.items():
#     pos = omgeo.position
#     ori = omgeo.orientation
# 
#     row = {
#         "string": omkey.string,
#         "om": omkey.om,
#         "pmt": getattr(omkey, "pmt", None),
# 
#         "x": pos.x,
#         "y": pos.y,
#         "z": pos.z,
# 
#         "dir_x": ori.dir.x,
#         "dir_y": ori.dir.y,
#         "dir_z": ori.dir.z,
# 
#         "up_x": ori.up.x,
#         "up_y": ori.up.y,
#         "up_z": ori.up.z,
# 
#         "right_x": ori.right.x,
#         "right_y": ori.right.y,
#         "right_z": ori.right.z,
# 
#         "omtype": int(omgeo.omtype),
#         "area": float(omgeo.area),
#     }
# 
#     # add all other simple attributes from omgeo
#     for name in attrs:
#         v = getattr(omgeo, name)
#         if isinstance(v, (int, float, str, bool)):
#             row[name] = v
# 
#     rows.append(row)
# 
# om_df = pd.DataFrame(rows)
# om_df.head()
# 

In [52]:
# len(om_df)

In [53]:
# # Columns that have the same value in all rows
# const_cols = om_df.columns[om_df.nunique(dropna=False) == 1]
# print(const_cols)
# 

In [54]:
# # those same values:
# print(om_df[const_cols].iloc[0])
# 

In [55]:
# om_df = om_df.loc[:, om_df.nunique(dropna=False) > 1]


In [56]:
# om_df

In [57]:
# om_df["pos_tuple"] = list(
#     zip(om_df["x"].round(6), om_df["y"].round(6), om_df["z"].round(6))
# )
# 
# uniq_per_om = om_df.groupby(["string", "om"])["pos_tuple"].nunique()
# 
# print("Max distinct positions per (string, om):", uniq_per_om.max())
# 
# if uniq_per_om.max() == 1:
#     print("=> Positions are OM-level (same for all PMTs within an OM).")
# else:
#     print("=> Positions are PMT-level (they vary between PMTs in the same OM).")
# 
# example_key = uniq_per_om.idxmax()
# print("\nExample (string, om):", example_key)
# print(om_df[(om_df["string"] == example_key[0]) & (om_df["om"] == example_key[1])])
# 

# - Subdetectors (same as Geometry.Subdetectors)¶


In [58]:
# subdets = detector_status["Subdetectors"]
# print(type(subdets))

In [59]:
# sample_key = next(iter(subdets.keys()))
# attrs = [a for a in dir(sample_key) if not a.startswith("_")]
# 
# rows = []
# 
# for mkey in subdets.keys():
#     name = subdets[mkey]   
# 
#     row = {"subdetector_name": str(name)}
# 
#     # add simple attributes from ModuleKey
#     for attr in attrs:
#         v = getattr(mkey, attr)
#         if isinstance(v, (int, float, str, bool)):
#             row[attr] = v
# 
#     rows.append(row)
# 
# subdet_df = pd.DataFrame(rows)
# subdet_df


In [60]:
# len(subdets)
# # number of OM

In [61]:
# print(subdets)
# (name of these subdetectors: Upgrade)

# ModuleKey has information of StringId and OM Id

# - DetectorStatus.BadDomsList
#### there is no any bad dom

In [62]:
bad_doms = detector_status["BadDomsList"]

print("Type:", type(bad_doms))

Type: <class 'icecube._dataclasses.I3VectorOMKey'>


In [63]:
try:
    print("Number of entries:", len(bad_doms))
except TypeError:
    print("len() not defined")

print("\nFirst few entries:")
for i, omkey in enumerate(bad_doms):
    print("  ", omkey)


Number of entries: 0

First few entries:


# - DetectorStatus.BadDomsListSLC
##### no info here

In [64]:
bad_doms_slc = detector_status["BadDomsListSLC"]

print("Type:", type(bad_doms_slc))

Type: <class 'icecube._dataclasses.I3VectorOMKey'>


In [65]:
try:
    print("Number of entries:", len(bad_doms_slc))
except TypeError:
    print("len() not defined")

print("\nFirst few entries:")
for i, omkey in enumerate(bad_doms_slc):
    print("  ", omkey)


Number of entries: 0

First few entries:
